##### Pacotes Necessários 

In [90]:
import psycopg2
import pandas as pd
from configparser import ConfigParser

##### Conexão com a Base de Dados

In [18]:
# Função para configuração de conexão com parâmetros de acesso presentes no arquivo database.ini
# Transforma o arquivo em uma variável de objeto
def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return db

In [67]:
def connect():
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()

        retorno = []

        cursor.execute('SELECT * FROM forms')
        forms = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'forms'")
        colunas = cursor.fetchall()
        retorno.append([forms, colunas])
        cursor.execute('SELECT * FROM orders')

        orders = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders'")
        colunas = cursor.fetchall()
        retorno.append([orders, colunas])

        cursor.execute('SELECT * FROM order_items')
        items = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'order_items'")
        colunas = cursor.fetchall()
        retorno.append([items, colunas])

        cursor.close()
        return retorno
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Conexão encerrada.')


if __name__ == '__main__':
    retorno = connect()

Conexão encerrada.


In [116]:
forms = pd.DataFrame(retorno[0][0]).dropna()
forms.columns = [item for sublist in retorno[0][1] for item in sublist]

orders = pd.DataFrame(retorno[1][0]).dropna()
orders.columns = [item for sublist in retorno[1][1] for item in sublist]

items = pd.DataFrame(retorno[2][0]).dropna()
items.columns = [item for sublist in retorno[2][1] for item in sublist]